In [28]:
# 143

## xml을 이용한 스크래핑

In [11]:
# ElementTree 모듈을 로드 -> XML의 파서 중 하나
from xml.etree import ElementTree

In [12]:
# 기상청에서 rss문서를 rss.xml로 받기
# http://www.weather.go.kr/weather/lifenindustry/sevice_rss.jsp
# rss문서 연 다음 우클릭 - 다른 이름으로 저장 - .xml

In [13]:
# parse() 함수로 파일을 읽고 ElementTree 객체를 생성
tree = ElementTree.parse('rss.xml')

In [14]:
# XML, HTML은 계층적(hierarchical) 구조이며 최상위를 root라고 부름
# getroot() 메서드로 XML의 루트 element를 추출
root = tree.getroot()  # root : 최상위.  getroot로 최상위 디렉토리를 찾음

In [15]:
import pandas as pd  # pandas : 데이터프레임 전용 라이브러리

In [16]:
데이터프레임_리스트 = []
for item in root.findall('channel/item/description/body/location/data'):  # findall : 루트부터 전체 디렉토리 찾기
    # find() 메서드로 element 탐색, text 속성으로 값을 추출
    tm_ef = item.find('tmEf').text  # 태그명을 확인하여 변수에 넣음
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    데이터프레임 = pd.DataFrame({     # 데이터프레임 형식으로 만듦
        '일시':[tm_ef],
        '최저기온':[tmn],
        '최고기온':[tmx],
        '날씨':[wf],
    })
    데이터프레임_리스트.append(데이터프레임)
날씨정보 = pd.concat(데이터프레임_리스트)  # concat을 이용해 여러행을 합침
날씨정보  

,일시,최저기온,최고기온,날씨
0,2021-07-17 00:00,24,32,구름많음
0,2021-07-17 12:00,24,32,구름많고 소나기
0,2021-07-18 00:00,24,33,맑음
0,2021-07-18 12:00,24,33,구름많음
0,2021-07-19 00:00,24,33,맑음
...,...,...,...,...
0,2021-07-21 00:00,23,32,맑음
0,2021-07-21 12:00,23,32,맑음
0,2021-07-22 00:00,23,31,구름많음
0,2021-07-23 00:00,23,32,맑음


In [17]:
type(날씨정보)

pandas.core.frame.DataFrame

In [18]:
날씨정보.to_csv('날씨정보.csv')

In [19]:
엑셀 = pd.ExcelWriter('날씨정보.xlsx')
날씨정보.to_excel(엑셀, '.', index=False )
엑셀.save()

In [20]:
날씨정보.reset_index(drop=True, inplace=True)

In [21]:
날씨정보.to_json('날씨정보.json')

In [22]:
import sqlite3
from pandas.io import sql
import os

In [23]:
with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
    try:
        날씨정보.to_sql(name = 'WEATHER_INFO', con = con, index = False, if_exists='append') 
        #if_exists : {'fail', 'replace', 'append'} default : fail
    except Exception as e:
        print(str(e))
    
    query = 'SELECT * FROM WEATHER_INFO'
    데이터프레임1 = pd.read_sql(query, con = con)

In [24]:
데이터프레임1

,일시,최저기온,최고기온,날씨
0,2021-07-17 00:00,24,32,구름많음
1,2021-07-17 12:00,24,32,구름많고 소나기
2,2021-07-18 00:00,24,33,맑음
3,2021-07-18 12:00,24,33,구름많음
4,2021-07-19 00:00,24,33,맑음
...,...,...,...,...
450,2021-07-21 00:00,23,32,맑음
451,2021-07-21 12:00,23,32,맑음
452,2021-07-22 00:00,23,31,구름많음
453,2021-07-23 00:00,23,32,맑음


In [25]:
엑셀 = pd.ExcelWriter('날씨정보2.xlsx')
데이터프레임1.to_excel(엑셀, '.', index=False )
엑셀.save()

In [26]:
df = pd.read_excel('날씨정보2.xlsx')

In [27]:
df

,일시,최저기온,최고기온,날씨
0,2021-07-17 00:00,24,32,구름많음
1,2021-07-17 12:00,24,32,구름많고 소나기
2,2021-07-18 00:00,24,33,맑음
3,2021-07-18 12:00,24,33,구름많음
4,2021-07-19 00:00,24,33,맑음
...,...,...,...,...
450,2021-07-21 00:00,23,32,맑음
451,2021-07-21 12:00,23,32,맑음
452,2021-07-22 00:00,23,31,구름많음
453,2021-07-23 00:00,23,32,맑음
